### 06.03. Import agents into application

In [ ]:
#Reuse the agents created in the previous chapters by importing their notebooks
#Ignore / hide the code output and use only the agents

#import the Healthcare assistant agent
%run "code_02_XX A Healthcare assistant agent with ReAct.ipynb" 
print("===============================================================")
#import the appointment scheduling agent
%run "code_05_XX ReAct agent with Workflows - Doctor scheduling.ipynb"

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    
> Running step ac8c995e-4db0-4cd8-94a9-a1b01c191b09. Step input: Which medication should i take for arthritis?
Thought: The current language of the user is: English. I need to use a tool to help me ans

### 06.04 Setup the multi-agent application

In [40]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.tools import FunctionTool

#Sync agent - use as query agent directly
healthcare_agent_tool = QueryEngineTool(
    query_engine= healthcare_agent,
    metadata=ToolMetadata(
        name="Healthcare_Agent",
        description="""
            An agent that can answer questions about diseases,
            symptoms for the diseases and the medications to cure them
            """
    )
)

#Async agent - use an async function to call
async def async_doctors_and_appointments(query: str) -> str:
    response = await scheduling_agent.run(input=query)
    return response.get("response")

#Use a sync wrapper over the async function
def doctors_and_appointments(query: str) -> str :
    """
    An agent that find doctors for specific diseases & specialities
    and then schedule appointments for these doctors.

    Inputs:
    query: This is the original query from the user
    """
    response=asyncio.run(async_doctors_and_appointments(query))
    return response

appointment_scheduling_tool = FunctionTool.from_defaults(doctors_and_appointments)
    
#Create a Multi-agent
multi_agent = ReActAgent.from_tools(
                [healthcare_agent_tool, appointment_scheduling_tool],
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


### 06.05. Execute the multi-agent application

In [46]:
response=multi_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 856bc6e9-c189-4c68-b05e-562b1ee2912d. Step input: Which medication should i take for arthritis?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: Healthcare_Agent
Action Input: {'input': 'medication for arthritis'}
> Running step 9357a40e-6d51-4342-bbb1-b356e81bd4e9. Step input: medication for arthritis
Thought: The current language of the user is English. I need to use a tool to help me answer the question about medications for arthritis.
Action: query_engine_tool
Action Input: {'input': 'arthritis'}
Observation: For arthritis, Naproxen is used, with a typical dosage of 250-500 mg taken twice daily.
> Running step 276ba925-7e03-4fb8-bacb-8bcdcf0e8d13. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: For arthritis, Naproxen is commonly used, with a typical dosage of 250-500 mg taken twice daily.
Observation: For arthritis, Naproxen is commo

In [45]:
response=multi_agent.chat("Which doctors are cardiologists?")
print("*******\n Response : ",response.response)

> Running step 69933fb1-ba08-40ac-9bd7-5010b291301d. Step input: Which doctors are cardiologists?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: doctors_and_appointments
Action Input: {'query': 'cardiologist'}
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
*** LLM Returned :  thought='(Implicit) I can answer without any more tools!' response='The cardiologist available is John Smith, who has 15 years of experience.' is_streaming=False
Step handle_llm_input produced event StopEvent
Observation: The cardiologist available is John Smith, who has 15 years of experience.
> Running step 9756407e-6b37-40b2-aa88-ee8cb51d9430. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The cardiologist available is John Smith, who has 1

In [44]:
response=multi_agent.chat(
    "Find a neurologist and request an appointment for Beth Wilson at the earliest")

> Running step 18607dda-7160-44ea-8aec-c1f9979ce09c. Step input: Find a neurologist and request an appointment for Beth Wilson at the earliest
Thought: The current language of the user is English. I need to use a tool to find a neurologist and schedule an appointment for Beth Wilson.
Action: doctors_and_appointments
Action Input: {'query': 'Find a neurologist and request an appointment for Beth Wilson at the earliest'}
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
*** LLM Returned :  thought='The current language of the user is English. I need to use a tool to find a neurologist.' action='query_engine_tool' action_input={'input': 'neurologist'}
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
*** Calling tools :  [ToolSelection(tool_id='fake', tool_name='query_engine_tool', tool_kwargs={'input': 'neurologist'})]
Step